# 모듈 설치

In [1]:
import os
import sys
# import re
import pickle
from collections import defaultdict, OrderedDict, Counter
import warnings
warnings.filterwarnings('ignore')

# import json
from tqdm import tqdm

In [2]:
import numpy as np
import pandas as pd
# from pandas.plotting import scatter_matrix
# from pandas_profiling import ProfileReport
# from missingno import dendrogram
# import missingno as msno

# pd.options.display.float_format = '{:0.5f}'.format`

In [3]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# settings for seaborn plot sizes
# sns.set(rc={'figure.figsize':(4.5, 3)})

----

# 데이터 수집 

## data 파일에 있는 csv 리스트 만들기 

In [4]:
import os

dir_path = "data"

path_ls = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        path_ls.append(file_path)

In [5]:
dff_dict= defaultdict(list)
for path in tqdm(path_ls):
    name = path.split('_')[-2]
    dff = pd.read_csv(f'{path}', header=None)
    dff_dict[name].append(dff)

100%|██████████| 68/68 [01:09<00:00,  1.02s/it]


In [ ]:
len(dff_dict)

19

In [ ]:
# 파일 확인
for key, value in dff_dict.items():
    t = '-'
    print(f'{t*20}{key}{t*20}')
    for idx, dff in enumerate(value):
        print(dff)
        print({'-'*20})
    print()
    print()
    print()

--------------------KGUINDTAMOUNT--------------------
                  0   1       2      3         4       5
0        2019-07-17   0     도시락    경기도      의정부시   21000
1        2019-07-17   0  돈까스/일식    경기도      의정부시   35000
2        2019-07-17   0  돈까스/일식   충청북도       제천시   42000
3        2019-07-17   0  배달전문업체    경기도  고양시 일산동구   23125
4        2019-07-17   0  배달전문업체    경기도      의정부시   16633
...             ...  ..     ...    ...       ...     ...
3223127  2021-06-30  23       회    경기도       이천시   34500
3223128  2021-06-30  23       회   경상북도       김천시   53250
3223129  2021-06-30  23       회  부산광역시        남구   20500
3223130  2021-06-30  23       회   전라남도       나주시   12750
3223131  2021-06-30  23       회   충청남도   천안시 동남구  317000

[3223132 rows x 6 columns]
{'--------------------'}



--------------------KGUINDTDLVR--------------------
                  0   1       2      3         4      5
0        2019-07-17   0     도시락    경기도      의정부시  13.52
1        2019-07-17   0  돈까스/일식    경기도    

- KGUINDTAMOUNT
    - 이상무
- KGUINDTDLVR
    - [-1]
- KGUINDTORDER
    - [-1]
- KGUPOPLTNINFO
    - 연결
- KGURESIDEINFO
    - 연결
- KGUTIMEAMOUNT
    - [-1]
- KGUTIMEDLVR
    - [-1]
- KGUTIMEORDER
    - [-1]
- KGUTRDARINFO
    - 연결
- KGUWETHERINFO
    - 연결
    - 앞의 끝은 20201031
    - 뒤의 시작은 20201109
    - 공백 존재
- KGUAREAAVDSTN
    - 연결
- KGUAREADOITEM
    - 연결
    - 날짜에 이상한 에러가 보임
        - ﻿2021-07-01
        - 확인 필요
- KGUAREAINDPAY
    - 연결
- KGUDSTNAMOUNT
    - 연결
- KGUDSTNORDER
    - 연결
- KGUINSCTNINFO
    - 특별취급 / 연결
    - 데이터 중에 칼럼 있는거 하고 없는게 있음
    - 결합후 데이터 확인 필요
        - ﻿2021
        - ﻿YEAR
- KGUINSCTNORDR
    - 연결
- KGUWTHRDLVRDF
    - 연결
- KGUDLIVERYSTORE00001
    - 배달 상점 데이터
    - 흠...
    - 결합하고 배달상점ID(cols[0]) 기준으로 duplicate 하고 최신 버전으로 간주되는 마지막꺼를 살리기

- [-1]
    - 버전이 올라갈수록 기존 버전을 포함하며 업데이트되는 형식
    - 마지막 버전만 최종데이터로 판단하기
    - 5개
    - ['KGUINDTDLVR', 'KGUINDTORDER', 'KGUTIMEAMOUNT', 'KGUTIMEDLVR', 'KGUTIMEORDER']
- 이상무
    - 버전이 하나만 존재
    - 1개
    - ['KGUINDTAMOUNT']
- 연결
    - 버전이 올라갈수록 기존 버전의 다음 내용들을 업데이트하는 형식
    - 데이터 결합
    - 11개
    - ['KGUPOPLTNINFO', 'KGURESIDEINFO', 'KGUTRDARINFO', 'KGUWETHERINFO', 'KGUAREAAVDSTN', 'KGUAREADOITEM', 'KGUAREAINDPAY', 'KGUDSTNAMOUNT', 'KGUDSTNORDER', 'KGUINSCTNORDR', 'KGUWTHRDLVRDF']
- 특별취급
    - 2개
    - KGUDLIVERYSTORE00001, KGUINSCTNINFO

## 같은 유형의 파일끼리 연결

In [23]:
concat_dff_dict = defaultdict()
for key, value in dff_dict.items():
    print(f'----------{key}----------')
    
    # 버전이 올라갈수록 기존 버전을 포함하며 업데이트되는 형식
    keys = ['KGUINDTDLVR', 'KGUINDTORDER', 'KGUTIMEAMOUNT', 'KGUTIMEDLVR', 'KGUTIMEORDER']
    if key in keys:
        dff = value[-1]
    
    # 버전이 올라갈수록 기존 버전의 다음 내용들을 업데이트하는 형식
    keys = ['KGUPOPLTNINFO', 'KGURESIDEINFO', 'KGUTRDARINFO', 'KGUWETHERINFO', \
        'KGUAREAAVDSTN', 'KGUAREADOITEM', 'KGUAREAINDPAY', 'KGUDSTNAMOUNT', \
            'KGUDSTNORDER', 'KGUINSCTNORDR', 'KGUWTHRDLVRDF']
    if key in keys:
        dff = pd.concat(value)
    
    # 버전이 하나만 존재
    keys = ['KGUINDTAMOUNT']
    if key in keys:
        dff = value[0]  

    # 특별 취급
    keys = ['KGUDLIVERYSTORE00001', 'KGUINSCTNINFO']
    if key in keys:
        # 칼럼이 존재하는 버전이 있고 없는 버전이 있음
        # 시점은 다 다른 버전이므로 칼럼 유무에 다르게 결합
        if key == 'KGUINSCTNINFO':
            df_ls = []
            for df in value:
                # print(str(df.iloc[0,0])[-1])
                try:        # 첫행 1열이 int 가능 하면 칼럼x
                    # print('try')
                    check = int(str(df.iloc[0,0])[-1])
                    df_ls.append(df)
                except:     # 첫행 1열이 int 불가능 하면 칼럼o
                    # print('except', str(df.iloc[0,0])[-1])
                    df_ls.append(df.drop(0, axis=0))
            dff = pd.concat(df_ls)
        # 연결후 배달 상점ID가 같은 중복 데이터가 있으면 마지막꺼 살리기
        if key == 'KGUDLIVERYSTORE00001':
            dff = pd.concat(dff_dict[key])
            dff.drop_duplicates(subset=list(dff.columns)[0], keep='last', inplace=True)
        
    concat_dff_dict[key] = (dff)

----------KGUINDTAMOUNT----------
----------KGUINDTDLVR----------
----------KGUINDTORDER----------
----------KGUPOPLTNINFO----------
----------KGURESIDEINFO----------
----------KGUTIMEAMOUNT----------
----------KGUTIMEDLVR----------
----------KGUTIMEORDER----------
----------KGUTRDARINFO----------
----------KGUWETHERINFO----------
----------KGUAREAAVDSTN----------
----------KGUAREADOITEM----------
----------KGUAREAINDPAY----------
----------KGUDSTNAMOUNT----------
----------KGUDSTNORDER----------
----------KGUINSCTNINFO----------
9
9
0
0
R
1
----------KGUINSCTNORDR----------
----------KGUWTHRDLVRDF----------
----------KGUDLIVERYSTORE00001----------


In [24]:
for code, dff in concat_dff_dict.items():
    print(f'{code} - {dff.shape}')

KGUINDTAMOUNT - (3223132, 6)
KGUINDTDLVR - (3333595, 6)
KGUINDTORDER - (3367316, 6)
KGUPOPLTNINFO - (220374, 8)
KGURESIDEINFO - (495000, 9)
KGUTIMEAMOUNT - (2165132, 6)
KGUTIMEDLVR - (2206465, 6)
KGUTIMEORDER - (2240807, 6)
KGUTRDARINFO - (159651, 10)
KGUWETHERINFO - (5626750, 15)
KGUAREAAVDSTN - (14606, 7)
KGUAREADOITEM - (26398236, 6)
KGUAREAINDPAY - (11023306, 6)
KGUDSTNAMOUNT - (3794787, 6)
KGUDSTNORDER - (3794787, 6)
KGUINSCTNINFO - (30103, 23)
KGUINSCTNORDR - (54754, 7)
KGUWTHRDLVRDF - (380741, 27)
KGUDLIVERYSTORE00001 - (83159, 11)


----

## 칼럼명 추가

In [25]:
all_dict = defaultdict(list)

In [26]:
text = '날씨별 배달 품목차이'
code = 'KGUWTHRDLVRDF'
ls = ['광역시도명',
'시군구명',
'날짜',
'시간대별 시간',
'강수 유형명',
'습도값',
'강수량 값',
'기온값',
'풍속값',
'바람강도 유형명',
'동쪽서쪽바람유형명',
'풍향값',
'풍향카테고리명',
'한식_배달건수',
'분식_배달건수',
'카페/디저트_배달건수',
'돈까스/일식_배달건수',
'회_배달건수',
'치킨_배달건수',
'피자_배달건수',
'아시안/양식_배달건수',
'중식_배달건수',
'족발/보쌈_배달건수',
'야식_배달건수',
'찜탕_배달건수',
'도시락_배달건수',
'패스트푸드_배달건수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [27]:
text = '지역-업종 배달평균거리'
code = 'KGUAREAAVDSTN'
ls = ['년도',
'월',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'배달주문수',
'배달평균거리값']
all_dict[text].append(code)
all_dict[text].append(ls)

In [28]:
text = '배달 상점 데이터'
code = 'KGUDLIVERYSTORE00001'
ls = ['배달상점ID'	,
'배달상점 업종명',
'배달상점 법정동코드'	,
'배달상점 시도명'	,
'배달상점 시군구명'	,
'배달상점 법정동 읍면동명'	,
'배달상점 법정동 리명'	,
'배달상점 행정동코드',
'배달상점 행정동 읍면동명'	,
'배달상점 도로명주소코드',
'배달상점 도로명주소 도로명']
all_dict[text].append(code)
all_dict[text].append(ls)

In [29]:
text = '업종 목적지별 배달 주문건수'
code = 'KGUDSTNORDER'
ls = ['날짜',
'시간대별 시간',
'배달상점 업종명',
'목적지 광역시도명',
'목적지 시군구명',
'주문 건수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [30]:
text = '업종-거리구간 배달 건수'
code = 'KGUINSCTNORDR'
ls = ['년도',
'월',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'배달거리 구간명',
'배달주문수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [31]:
text = '업종 목적지별 평균주문금액'
code = 'KGUDSTNAMOUNT'
ls = ['날짜',
'시간대별 시간',
'배달상점 업종명',
'목적지 광역시도명',
'목적지 시군구명',
'평균주문금액']
all_dict[text].append(code)
all_dict[text].append(ls)

In [32]:
text = '주문지역 주거 특성'
code = 'KGURESIDEINFO'
ls = ['기준연도',	'시군구코드',	'시도명',	'시군구명',	'세대구성대분류',	'세대구성중분류',	'세대구성소분류',	'단위','빈도수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [33]:
text = '주문지역 상권유형 특성'
code = 'KGUTRDARINFO'
ls = ['기준연도',	'시도명',	'시도코드',	'시군구명'	,'시군구코드'	,'상권업종대분류명',	'상권업종중분류명'	,'상권업종소분류명'	,'상권업종소분류코드'	,'상권업종개수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [34]:
text = '주문지역 인구 특성'
code = 'KGUPOPLTNINFO'
ls = ['기준연월',	'시군구코드',	'시도명',	'시군구명'	,'5세단위구분'	,'총인구수'	,'남성인구수','여성인구수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [35]:
text = '주문지역 기상 정보'
code = 'KGUWETHERINFO'
ls = ['순번',	'시군구코드',	'광역시도명',	'시군구명',	'공개 일',	'공개 시간',\
    '강수 유형명',	'습도',	'1시간 강수량 값',	'기온',	'풍속',	\
    '바람강도 유형명',	'동쪽서쪽바람유형명',	'풍향값',	'풍향카테고리명']
all_dict[text].append(code)
all_dict[text].append(ls)

In [36]:
text = '지역-업종 결제정보'
code = 'KGUAREAINDPAY'
ls = ['날짜',
'24시기준 해당시간',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'결제종류값']
all_dict[text].append(code)
all_dict[text].append(ls)

In [37]:
text = '지역-요일 배달 품목정보'
code = 'KGUAREADOITEM'
ls = ['날짜',
'24시기준 해당시간',
'주문요일명',
'배달상점 광역시도명',
'배달상점 시군구명',
'배달상점 업종명']
all_dict[text].append(code)
all_dict[text].append(ls)

In [38]:
text = '업종-거리구간 상권 정보'
code = 'KGUINSCTNINFO'
ls = ['년도',
'월',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'1km이하 배달건수',
'1km초과3km이하배달건수',
'3km초과5km이하배달건수',
'거리 5000이상',
'한식_매장수',
'분식_매장수',
'카페/디저트_매장수',
'돈까스/일식_매장수',
'회_매장수',
'치킨_매장수',
'피자_매장수',
'아시안/양식_매장수',
'중식_매장수',
'족발/보쌈_매장수',
'야식_매장수',
'찜탕_매장수',
'도시락_매장수',
'패스트푸드_매장수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [39]:
text = '시간-지역별 배달 주문건수'
code = 'KGUTIMEORDER'
ls = ['날짜',	'시간대별 시간',	'배달상점 광역시도명',	'배달상점 시군구명',	'배달상점 법정동명',	'주문건수']
all_dict[text].append(code) 
all_dict[text].append(ls)

In [40]:
text = '시간-지역별 배달 평균주문금액'
code = 'KGUTIMEAMOUNT'
ls = ['날짜',	'시간대별 시간',	'배달상점 광역시도명',	\
    '배달상점 시군구명',	'배달상점 법정동명',	'평균주문금액']
all_dict[text].append(code) 
all_dict[text].append(ls)

In [41]:
text = '시간-지역별 배달 소요시간평균'
code = 'KGUTIMEDLVR'
ls = ['날짜',	'시간대별 시간',	'배달상점 업종명',\
    '배달상점 광역시도명',    '배달상점 시군구명',	'평균배달소요시간']
all_dict[text].append(code) 
all_dict[text].append(ls)

In [42]:
text = '업종-지역별 평균주문금액'
code = 'KGUINDTAMOUNT'
ls = ['날짜',	'시간대별 시간',	'배달상점 업종명',	'배달상점 광역시도명',	'배달상점 시군구명',	'평균주문금액']
all_dict[text].append(code)
all_dict[text].append(ls)

In [43]:
text = '업종-지역별 배달 주문건수'
code = 'KGUINDTORDER'
ls = ['날짜',	'시간대별 시간',	'배달상점 업종명',	'배달상점 광역시도명',	'배달상점 시군구명',	'주문 건수']
all_dict[text].append(code) 
all_dict[text].append(ls)

In [44]:
text = '업종-지역별 평균배달소요시간'
code = 'KGUINDTDLVR'
ls = ['날짜',	'시간대별 시간',	'배달상점 업종명',	'배달상점 광역시도명',	'배달상점 시군구명',	'평균배달소요시간(분)']
all_dict[text].append(code) 
all_dict[text].append(ls)

In [45]:
concat_dff_codes = set(concat_dff_dict.keys())
check_codes = set()
for name, ls in all_dict.items():
    # print(name)
    code = ls[0]
    if code in concat_dff_codes:
        check_codes.add(code)
print(concat_dff_codes - check_codes)

set()


- 모든 데이터에 대한 칼럼명 리스트 생성 확인

In [46]:
for name, ls in all_dict.items():
    # print(name)
    code = ls[0]
    cols = ls[1]
    try:
        # print('성공')
        concat_dff_dict[code].columns = cols
    except:
        print(name)
        print(cols)
        print(concat_dff_dict[code].columns)
        print()
    # cols = ls[1]
    # concat_dff_dict[code].columns = cols
    

In [47]:
for code, dff in concat_dff_dict.items():
    print(code)
    print(dff.shape)
    print(dff.columns)
    print()

KGUINDTAMOUNT
(3223132, 6)
Index(['날짜', '시간대별 시간', '배달상점 업종명', '배달상점 광역시도명', '배달상점 시군구명', '평균주문금액'], dtype='object')

KGUINDTDLVR
(3333595, 6)
Index(['날짜', '시간대별 시간', '배달상점 업종명', '배달상점 광역시도명', '배달상점 시군구명', '평균배달소요시간(분)'], dtype='object')

KGUINDTORDER
(3367316, 6)
Index(['날짜', '시간대별 시간', '배달상점 업종명', '배달상점 광역시도명', '배달상점 시군구명', '주문 건수'], dtype='object')

KGUPOPLTNINFO
(220374, 8)
Index(['기준연월', '시군구코드', '시도명', '시군구명', '5세단위구분', '총인구수', '남성인구수', '여성인구수'], dtype='object')

KGURESIDEINFO
(495000, 9)
Index(['기준연도', '시군구코드', '시도명', '시군구명', '세대구성대분류', '세대구성중분류', '세대구성소분류', '단위',
       '빈도수'],
      dtype='object')

KGUTIMEAMOUNT
(2165132, 6)
Index(['날짜', '시간대별 시간', '배달상점 광역시도명', '배달상점 시군구명', '배달상점 법정동명', '평균주문금액'], dtype='object')

KGUTIMEDLVR
(2206465, 6)
Index(['날짜', '시간대별 시간', '배달상점 업종명', '배달상점 광역시도명', '배달상점 시군구명', '평균배달소요시간'], dtype='object')

KGUTIMEORDER
(2240807, 6)
Index(['날짜', '시간대별 시간', '배달상점 광역시도명', '배달상점 시군구명', '배달상점 법정동명', '주문건수'], dtype='object')

KGUTRDARINFO
(159651, 10)
Index

## 파일로 저장

In [48]:
for i, (name, ls) in enumerate(all_dict.items()):
    # print(name)
    code = ls[0]
    cols = ls[1]
    print(i, name, code)
    concat_dff_dict[code].to_csv(f'type_data/{name}.csv', index=False)

0 날씨별 배달 품목차이 KGUWTHRDLVRDF
1 지역-업종 배달평균거리 KGUAREAAVDSTN
2 배달 상점 데이터 KGUDLIVERYSTORE00001
3 업종 목적지별 배달 주문건수 KGUDSTNORDER
4 업종-거리구간 배달 건수 KGUINSCTNORDR
5 업종 목적지별 평균주문금액 KGUDSTNAMOUNT
6 주문지역 주거 특성 KGURESIDEINFO
7 주문지역 상권유형 특성 KGUTRDARINFO
8 주문지역 인구 특성 KGUPOPLTNINFO
9 주문지역 기상 정보 KGUWETHERINFO
10 지역-업종 결제정보 KGUAREAINDPAY
11 지역-요일 배달 품목정보 KGUAREADOITEM
12 업종-거리구간 상권 정보 KGUINSCTNINFO
13 시간-지역별 배달 주문건수 KGUTIMEORDER
14 시간-지역별 배달 평균주문금액 KGUTIMEAMOUNT
15 시간-지역별 배달 소요시간평균 KGUTIMEDLVR
16 업종-지역별 평균주문금액 KGUINDTAMOUNT
17 업종-지역별 배달 주문건수 KGUINDTORDER
18 업종-지역별 평균배달소요시간 KGUINDTDLVR


In [49]:
DataBase_info = {}
for name, ls in tqdm(all_dict.items()):
    # print(name)
    code = ls[0]
    cols = ls[1]
    dff = concat_dff_dict[code]
    info = {'name': name,
           'code': code,
           'len': len(dff),
           'columns': list(dff.columns),
           'shape': dff.shape}
    DataBase_info[name] = info

100%|██████████| 19/19 [00:00<00:00, 6350.95it/s]


In [50]:
with open('Database_info.p', 'wb') as file:    # james.p 파일을 바이너리 쓰기 모드(wb)로 열기
    pickle.dump(DataBase_info, file)

In [51]:
DataBase = {}
for name, ls in tqdm(all_dict.items()):
    # print(name)
    code = ls[0]
    cols = ls[1]
    dff = concat_dff_dict[code]
    info = {'name': name,
           'code': code,
           'len': len(dff),
           'columns': list(dff.columns),
           'shape': dff.shape}
    DataBase[name] = {'info': info, 'data': dff} 

100%|██████████| 19/19 [00:00<00:00, 12202.08it/s]


In [52]:
with open('Database.p', 'wb') as file:    # james.p 파일을 바이너리 쓰기 모드(wb)로 열기
    pickle.dump(DataBase, file)